In [40]:
import os
import random
import re
import openpyxl
import csv
import pandas as pd

jumpWords = ['UN']
#轉換資料所需暫存變數
playState = -1 #0:遊戲初始化階段 1:打牌階段 2:和牌 3:流局
startPlayer = 0
eventState = 0 #0:nothing 1:REACH 2:N 3:DORA 4:INIT DORA
eventPlayer = -1
windlist = [0,1,2,3]#0:東 1:南 2:西 3:北 #
Field_wind = 0
score = [250,250,250,250]#
dora = [-1,-1,-1,-1,-1] #
hai0 = []#
hai1 = []#
hai2 = []#
hai3 = []#
N0 = []#
N1 = []#
N2 = []#
N3 = []#
rounds = 0#
play1Dis = []
play2Dis = []
play3Dis = []
play4Dis = []
IfREACHPlayer = [0,0,0,0] #
IfPlayerChow = [0,0,0,0]
IfPlayerPung = [0,0,0,0]
IfPlayerKong = [0,0,0,0]
datalist = []
rowData = [0]*35
#注意 剛加入還是可以透過rowData改datalist的值
cur_game = 1
total_games = 100000

In [41]:
#函數宣告集中處
def GameResetValue():
    global IfPlayerChow,IfPlayerPung,IfPlayerKong,playState,windlist,score,dora,hai0,hai1,hai2,hai3,N0,N1,N2,N3,rounds,play1Dis,play2Dis,play3Dis,play4Dis,rowData,IfREACHPlayer,eventPlayer,eventState,startPlayer
    playState = -1
    startPlayer = 0
    eventPlayer = -1
    eventState = 0
    score = [250,250,250,250]
    dora = [-1,-1,-1,-1,-1]
    hai0 = []
    hai1 = []
    hai2 = []
    hai3 = []
    N0 = []
    N1 = []
    N2 = []
    N3 = []
    rounds = 0
    play1Dis = []
    play2Dis = []
    play3Dis = []
    play4Dis = []
    IfREACHPlayer = [0,0,0,0]
    IfPlayerChow = [0,0,0,0]
    IfPlayerPung = [0,0,0,0]
    IfPlayerKong = [0,0,0,0]
    
def intTo16bitBinary(x):#int轉2進制16位
    binaryStr = ''
    for i in range(15,-1,-1):
        bit = (x >> i) & 1
        binaryStr += str(bit)
    return binaryStr

def Binary16bitToInt(str_x,bits):#2進制16位轉int
    x_bits = str_x[:bits]
    decimal_value = int(x_bits, 2)
    return decimal_value

def haitrans(haiX,strlist):#初始化手牌
    haiX.clear()
    hailist = re.split(r'[",]', strlist)
    hailist = [s for s in hailist if s]
    del hailist[0]
    for tile in hailist:
        haiX.append(int(tile))
        
def chow(binary16Bit,eventPlayer):
    #print("chow")
    #print(binary16Bit)
    chowNumber = Binary16bitToInt(binary16Bit,6)
    tileType = int(chowNumber / 21) #0:character 1:circle 2:bamboo
    chowType = int(chowNumber / 3) % 7#0~6種
    chowTile = chowNumber % 3 #0最小 1中間 2最大
    smallTile =  tileType * 9 + chowType #吃的最小牌
    IfPlayerChow[eventPlayer] = 1
    if(eventPlayer == 0):
        N0.append(smallTile)
        N0.append(smallTile + 1)
        N0.append(smallTile + 2)
        hai0.append(smallTile + chowTile)
    elif(eventPlayer == 1):
        N1.append(smallTile)
        N1.append(smallTile + 1)
        N1.append(smallTile + 2)
        hai1.append(smallTile + chowTile)
    elif(eventPlayer == 2):
        N2.append(smallTile)
        N2.append(smallTile + 1)
        N2.append(smallTile + 2)
        hai2.append(smallTile + chowTile)
    elif(eventPlayer == 3):
        N3.append(smallTile)
        N3.append(smallTile + 1)
        N3.append(smallTile + 2)
        hai3.append(smallTile + chowTile)
    
def pung(binary16Bit,eventPlayer):
    #print("pung")
    #print(binary16Bit)
    pungNumber = Binary16bitToInt(binary16Bit,7)
    tileNumber = int(pungNumber / 3)
    IfPlayerPung[eventPlayer] = 1
    if(eventPlayer == 0):
        N0.append(tileNumber)
        N0.append(tileNumber)
        N0.append(tileNumber)
        hai0.append(tileNumber)
    elif(eventPlayer == 1):
        N1.append(tileNumber)
        N1.append(tileNumber)
        N1.append(tileNumber)
        hai1.append(tileNumber)
    elif(eventPlayer == 2):
        N2.append(tileNumber)
        N2.append(tileNumber)
        N2.append(tileNumber)
        hai2.append(tileNumber)
    elif(eventPlayer == 3):
        N3.append(tileNumber)
        N3.append(tileNumber)
        N3.append(tileNumber)
        hai3.append(tileNumber)
    
def kong(binary16Bit,eventPlayer):
    #print("kong")
    #print(binary16Bit)
    IfPlayerKong[eventPlayer] = 1
    if(binary16Bit[11] == '0'): #暗槓
        kongNumber = Binary16bitToInt(binary16Bit,8)
        tileNumber = int(kongNumber / 4)
        if(eventPlayer == 0):
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
        elif(eventPlayer == 1):
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
        elif(eventPlayer == 2):
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
        elif(eventPlayer == 3):
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
    else:#明槓
        kongNumber = Binary16bitToInt(binary16Bit,7)
        tileNumber = int(kongNumber / 3)
        if(eventPlayer == 0):
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
            N0.append(tileNumber)
            if(binary16Bit[14] != '0' and binary16Bit[15] != '0'):
                hai0.append(tileNumber)
        elif(eventPlayer == 1):
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
            N1.append(tileNumber)
            if(binary16Bit[14] != '0' and binary16Bit[15] != '0'):
                hai1.append(tileNumber)
        elif(eventPlayer == 2):
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
            N2.append(tileNumber)
            if(binary16Bit[14] != '0' and binary16Bit[15] != '0'):
                hai2.append(tileNumber)
        elif(eventPlayer == 3):
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
            N3.append(tileNumber)
            if(binary16Bit[14] != '0' and binary16Bit[15] != '0'):
                hai3.append(tileNumber)
    
def playMode(element, player_pos, writer):#進入打牌階段
    global playState,windlist,score,dora,hai0,hai1,hai2,hai3,N0,N1,N2,N3,rounds,play1Dis,play2Dis,play3Dis,play4Dis,rowData,IfREACHPlayer,eventPlayer,eventState
    #print(element)
    #print(eventState, eventPlayer)
    if(playState == 2 or playState == 3):
        if('sc' in element):
            sclist = re.split(r'[",]', element)
            scnlist = [s for s in sclist if s]
            del scnlist[0]
            for i in range(len(score)):
                score[i] = int(scnlist[i * 2]) + int(scnlist[i * 2 + 1])
            write_to_excel(player_pos, writer)
    elif(element == 'DORA'):
        eventState = 3
    elif(eventState == 3):
        DORAhai = int(int(element.split('=')[1].strip('"'))/4)
        for i in range(len(dora)):
            if(dora[i] == -1):
                dora[i] = DORAhai
                break
        eventState = 0
    elif(element[0] == 'T'):
        rounds = rounds + 1
        hai0.append(int(element[1:]))
        #if player_pos == 0:
            #record_round(player_pos)
    elif(element[0] == 'D'):
        write_discard(writer, int(element[1:]), hai0)
        hai0.remove(int(element[1:]))
        play1Dis.append(int(element[1:]))
        #if player_pos != 0:
            #record_round(player_pos)
    elif(element[0] == 'U'):
        rounds = rounds + 1
        hai1.append(int(element[1:]))
        #if player_pos == 1:
            #record_round(player_pos)
    elif(element[0] == 'E'):
        write_discard(writer, int(element[1:]), hai1)
        hai1.remove(int(element[1:]))
        play2Dis.append(int(element[1:]))
        #if player_pos != 1:
            #record_round(player_pos)
    elif(element[0] == 'V'):
        rounds = rounds + 1
        hai2.append(int(element[1:]))
        #if player_pos == 2:
            #record_round(player_pos)
    elif(element[0] == 'F'):
        write_discard(writer, int(element[1:]), hai2)
        hai2.remove(int(element[1:]))
        play3Dis.append(int(element[1:]))
        #if player_pos != 2:
            #record_round(player_pos)
    elif(element[0] == 'W'):
        rounds = rounds + 1
        hai3.append(int(element[1:]))
        #if player_pos == 3:
            #record_round(player_pos)
    elif(element[0] == 'G'):
        write_discard(writer, int(element[1:]), hai3)
        hai3.remove(int(element[1:]))
        play4Dis.append(int(element[1:]))
        #if player_pos != 3:
            #record_round(player_pos)
    elif(element == 'AGARI'):
        #print("AGARI")
        playState = 2
        eventState = 0
    elif(element == 'RYUUKYOKU'):
        playState = 3
        eventState = 0
    elif(element == 'REACH'):
        eventState = 1
    elif(element == 'N'):
        eventState = 2
    elif(eventState == 1):
        if('who' in element):
            eventPlayer = int(element[5])
            IfREACHPlayer[eventPlayer] = 1
        elif('ten' in element):
            tenlist = re.split(r'[",]', element)
            tenlist = [s for s in tenlist if s]
            del tenlist[0]
            for i in range(len(score)):
                score[i] = int(tenlist[i])
        elif('step' in element and element[6] == '2'):
            eventPlayer = -1
            eventState = 0
    elif (eventState == 2 and eventPlayer != -1):
        Ntype = int(element.split('=')[1].strip('"'))
        binary16Bit = intTo16bitBinary(Ntype)
        if(binary16Bit[12] == '1'):
            pung(binary16Bit,eventPlayer)
        elif(binary16Bit[13] == '1'):
            chow(binary16Bit,eventPlayer)
        elif(binary16Bit[12] == '0' and binary16Bit[13] == '0'):
            kong(binary16Bit,eventPlayer)
        eventState = 0
        eventPlayer = -1
    elif(eventState == 2):
        eventPlayer = int(element[5])
        
def temproPrecess(temlist):
    for i in range(len(temlist)):
        temlist[i] = int(temlist[i] / 4)
    return sorted(temlist)

def test():
    print("Round: ",rounds)
    print("hai0: ",sorted(hai0))
    print("P0 Dis: ",play1Dis)
    print("N0: ",N0)
    print("hai1: ",sorted(hai1))
    print("P1 Dis: ",play2Dis)
    print("N1: ",N1)
    print("hai2: ",sorted(hai2))
    print("P2 Dis: ",play3Dis)
    print("N2: ",N2)
    print("hai3: ",sorted(hai3))
    print("P3 Dis: ",play4Dis)
    print("N3: ",N3)
    print("REACH: ", IfREACHPlayer)
    print("Chow: ",IfREACHPlayer)
    print("Pung: ",IfPlayerPung)
    print("Know: ",IfPlayerKong)
    print("score: ",score)
    print("dora: ",dora)
    print()
    
def record_round(player_pos):#紀錄該回合玩家可獲得資訊
    global datalist,rowData,Field_wind,playState,windlist,score,dora,hai0,hai1,hai2,hai3,N0,N1,N2,N3,rounds,play1Dis,play2Dis,play3Dis,play4Dis,IfREACHPlayer,eventPlayer,eventState
    rowData = [0] * 35
    rowData[0] = Field_wind
    rowData[1] = windlist[player_pos]
    rowData[2] = rounds
    
    hai34list = [0] * 34
    right34list = [0] * 34
    Opposite34list = [0] * 34
    Left34list = [0] * 34
    
    if player_pos == 0:
        hai34list = tileNumberTrans(hai0, hai34list)
        right34list = tileNumberTrans(play2Dis, right34list)
        Opposite34list = tileNumberTrans(play3Dis, Opposite34list)
        Left34list = tileNumberTrans(play4Dis, Left34list)
    elif player_pos == 1:
        hai34list = tileNumberTrans(hai1, hai34list)
        right34list = tileNumberTrans(play3Dis, right34list)
        Opposite34list = tileNumberTrans(play4Dis, Opposite34list)
        Left34list = tileNumberTrans(play1Dis, Left34list)
    elif player_pos == 2:
        hai34list = tileNumberTrans(hai2, hai34list)
        right34list = tileNumberTrans(play4Dis, right34list)
        Opposite34list = tileNumberTrans(play1Dis, Opposite34list)
        Left34list = tileNumberTrans(play2Dis, Left34list)
    elif player_pos == 3:
        hai34list = tileNumberTrans(hai3, hai34list)
        right34list = tileNumberTrans(play1Dis, right34list)
        Opposite34list = tileNumberTrans(play2Dis, Opposite34list)
        Left34list = tileNumberTrans(play3Dis, Left34list)
        
    cpoylist(3,hai34list)
    rowData[37] = score[player_pos]
    cpoylist(38,right34list)
    rowData[72] = score[(player_pos + 1) % 4]
    cpoylist(73,Opposite34list)
    rowData[107] = score[(player_pos + 2) % 4]
    cpoylist(108,Left34list)
    rowData[142] = score[(player_pos + 3) % 4]
    cpoylist(143,IfREACHPlayer)
    cpoylist(147, dora)
    rowData[152] = score[player_pos]
    datalist.append(rowData)
    
def tileNumberTrans(playerHai,templist):#將136張牌轉換成34種
    templist = [0] * 34
    for tile in playerHai:
        templist[int((tile/4))] = templist[int((tile/4))] + 1
    return templist

def cpoylist(RowIndex,copyedlist):#輸入rowData下一個空的index，把copyedlist的東西加到後面
    global rowData
    for data in copyedlist:
        rowData[RowIndex] = data
        RowIndex = RowIndex + 1

def write_to_excel(player_pos,writer):#輸入玩家視角，寫入excel
    global rowData, datalist, score
    #for rowIndex in range(len(datalist)):
        #datalist[rowIndex][-1] = score[player_pos]
    writer.writerows(datalist)
    datalist = []

def write_discard(writer, discard, hai):
    global rowData, datalist
    hai34 = [0]*34
    hai34 = tileNumberTrans(hai,hai34)
    cpoylist(0,hai34)
    rowData[34] = int(discard/4)
    datalist.append(rowData)
    rowData = [0]*35

In [ ]:
"""
將牌局資料轉成csv形式
Prevailing wind: 1東 2南 3西 4北
"""

for year in range(2021,2023):
    input_folder = f'E:/專題/data/{year}/scc game'
    with open(f'E:/專題/data/{year}/DiscardDataAll.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # 讀取第一個 CSV 檔案，假設其第一列的名子存放在第一列
        df1 = pd.read_excel('E:/專題/Discard_feature標籤值.xlsx', header=None)
        names = df1.iloc[0, :153].values  # 提取前153個名子
        pd.DataFrame([names]).to_csv(file, header=False, index=False)

        for filename in os.listdir(input_folder):
            cur_game = cur_game + 1

            GameResetValue()
            txt_path = os.path.join(input_folder, filename)
            random_Player = random.randint(0,3)
            windlist = [0,1,2,3]
            with open(txt_path, 'r', encoding='utf-8') as file:
                content = file.read()
                conlist = re.split(r'[<>]', content)
                conlist = [s for s in conlist if s]
                del conlist[1]
                newlist = []
                for item in conlist:
                    sub = re.split(r'[ /]', item)
                    for ele in sub:
                        newlist.append(ele);
                newlist = [s for s in newlist if s]
                GameResetValue()

                for element in newlist:
                    if(element in jumpWords):#不明原因在打牌階段出現的詞
                        continue

                    #初始手牌資訊
                    if(element == 'INIT'):
                        GameResetValue()
                        playState = 0
                        eventState = 4
                    elif(eventState == 4):
                        eventState = 0
                        number_str = element.split('=')[1].strip('"')
                        numbers = number_str.split(',')
                        Field_wind = int(numbers[0])
                        last_value = numbers[-1]
                        DORAhai = int(int(last_value) / 4)
                        for i in range(len(dora)):
                            if(dora[i] == -1):
                                dora[i] = DORAhai
                                break
                    elif(playState == 0 and 'oya' in element):
                        startPlayer = int(element[5])
                        for i in range(len(windlist)):
                            windlist[i] = (windlist[i] + 4 - startPlayer) % 4
                    elif(playState == 0 and 'ten' in element):
                        tenlist = re.split(r'[",]', element)
                        tenlist = [s for s in tenlist if s]
                        del tenlist[0]
                        for i in range(len(score)):
                            score[i] = int(tenlist[i])
                    elif(playState == 1 or playState == 2 or playState == 3):
                        playMode(element,random_Player,writer)
                    elif('hai0' in element):
                        haitrans(hai0, element)
                    elif('hai1' in element):
                        haitrans(hai1, element)
                    elif('hai2' in element):
                        haitrans(hai2, element)
                    elif('hai3' in element and playState == 0):
                        haitrans(hai3, element)
                        playState = 1